In [9]:
!pip install ohmysportsfeedspy

In [11]:
!pip install simplejson

  Using cached https://files.pythonhosted.org/packages/1a/47/1f37f183492f2d4f8b9ad2a2456caf6bd1e721bcd6f234acda0a51877447/simplejson-3.16.0-cp36-cp36m-win_amd64.whl


In [44]:
!pip install MySQLdb

  Could not find a version that satisfies the requirement MySQLdb (from versions: )
No matching distribution found for MySQLdb


In [45]:
#Import libraries
import pandas as pd
import json
import csv
from pandas.io.json import json_normalize
from ohmysportsfeedspy import MySportsFeeds
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()


#Connecting to website for our data
msf = MySportsFeeds(version="2.0")

msf.authenticate("d337138c-84ad-4657-94aa-ba4fa1", "MYSPORTSFEEDS")

In [2]:
#Pulling in CSV
output = msf.msf_get_data(league='nhl',season='2016-playoff',feed='player_injuries',format='csv')
print(output)

[b'#Date/Time of Update: 2018-12-11 02:12:19 GMT,#Player ID,#LastName,#FirstName,#Jersey Num,#Position,#Team ID,#Team Abbr.,#Team City,#Team Name,#Injury', b',14435,Anderson,Joey,49,RW,7,NJD,New Jersey,Devils,broken ankle (Out)', b',4620,Anisimov,Artem,15,C,20,CHI,Chicago,Blackhawks,concussion (Questionable)', b',5505,Armia,Joel,40,RW,14,MTL,Montreal,Canadiens,right knee (Out)', b',5538,Arvidsson,Viktor,33,LW,18,NSH,Nashville,Predators,broken thumb (Out)', b',4609,Backlund,Mikael,11,C,23,CGY,Calgary,Flames,concussion (Out)', b',4998,Baertschi,Sven,47,LW,21,VAN,Vancouver,Canucks,concussion (Out)', b',5205,Barberio,Mark,44,D,22,COL,Colorado,Avalanche,upper body (Out)', b',78,Bergeron,Patrice,37,C,11,BOS,Boston,Bruins,rib (Out)', b',5201,Bjugstad,Nick,27,C,4,FLO,Florida,Panthers,upper body (Questionable)', b',5067,Borowiecki,Mark,74,D,13,OTT,Ottawa,Senators,upper body (Out)', b',11807,Brodzinski,Jonny,17,C,28,LAK,Los Angeles,Kings,shoulder (Out)', b',10075,Buchnevich,Pavel,89,LW,9,NYR,New

In [3]:
#Creating dataframe
csv_file = "results/nhl_2016.csv"
nhl_2016 = pd.read_csv(csv_file)
nhl_2016.head()

,#Date/Time of Update: 2018-12-10 01:15:00 GMT,#Player ID,#LastName,#FirstName,#Jersey Num,#Position,#Team ID,#Team Abbr.,#Team City,#Team Name,#Injury
0,NaN,14435,Anderson,Joey,49.0,RW,7,NJD,New Jersey,Devils,broken ankle (Out)
1,NaN,5505,Armia,Joel,40.0,RW,14,MTL,Montreal,Canadiens,right knee (Out)
2,NaN,5538,Arvidsson,Viktor,33.0,LW,18,NSH,Nashville,Predators,broken thumb (Out)
3,NaN,4609,Backlund,Mikael,11.0,C,23,CGY,Calgary,Flames,upper body (Questionable)
4,NaN,4998,Baertschi,Sven,47.0,LW,21,VAN,Vancouver,Canucks,concussion (Out)


In [5]:
#Renaming columns
injury_2016_new= nhl_2016.rename(columns={"#LastName": "Last_Name",
                                         "#FirstName" : "First_Name",
                                         "#Jersey Num" : "Jersey_Number",
                                         "#Position" : "Team_Position",
                                         "#Team City": "Team_City",
                                         "#Team Name" : "Team_Name",
                                          "#Injury" : "Injury"})
#Putting together condensed DF
condensed_injury =injury_2016_new[["Last_Name", "First_Name", "Jersey_Number", "Team_Position", "Team_City", "Team_Name", "Injury"]]
condensed_injury.head()

,Last_Name,First_Name,Jersey_Number,Team_Position,Team_City,Team_Name,Injury
0,Anderson,Joey,49.0,RW,New Jersey,Devils,broken ankle (Out)
1,Armia,Joel,40.0,RW,Montreal,Canadiens,right knee (Out)
2,Arvidsson,Viktor,33.0,LW,Nashville,Predators,broken thumb (Out)
3,Backlund,Mikael,11.0,C,Calgary,Flames,upper body (Questionable)
4,Baertschi,Sven,47.0,LW,Vancouver,Canucks,concussion (Out)


In [6]:
#Pulling in JSON
output_2 = msf.msf_get_data(league='nhl',season='2016-playoff',feed='players',format='json')
print(output_2).prettify()

{'lastUpdatedOn': '2018-12-11T02:13:52.624Z', 'players': [{'player': {'id': 5263, 'firstName': 'Spencer', 'lastName': 'Abbott', 'primaryPosition': 'LW', 'alternatePositions': [], 'jerseyNumber': 24, 'currentTeam': None, 'currentRosterStatus': 'RETIRED', 'currentInjury': None, 'height': '5\'9"', 'weight': 170, 'birthDate': '1988-04-30', 'age': 30, 'birthCity': 'Hamilton, ON', 'birthCountry': 'Canada', 'rookie': False, 'highSchool': None, 'college': None, 'handedness': {'shoots': 'R'}, 'officialImageSrc': 'https://nhl.bamcontent.com/images/headshots/current/168x168/8476805@2x.jpg', 'socialMediaAccounts': [], 'currentContractYear': None, 'drafted': None, 'externalMappings': [{'source': 'NHL.com', 'id': 8476805}]}, 'teamAsOfDate': None}, {'player': {'id': 4419, 'firstName': 'Justin', 'lastName': 'Abdelkader', 'primaryPosition': 'LW', 'alternatePositions': [], 'jerseyNumber': 8, 'currentTeam': {'id': 16, 'abbreviation': 'DET'}, 'currentRosterStatus': 'ROSTER', 'currentInjury': None, 'height

AttributeError: 'NoneType' object has no attribute 'prettify'

In [7]:
#Drilling down to get info we need
players = output_2["players"]

In [11]:
#Seeing headers for specific calls
players[0]["player"][]

{'id': 5263,
 'firstName': 'Spencer',
 'lastName': 'Abbott',
 'primaryPosition': 'LW',
 'alternatePositions': [],
 'jerseyNumber': 24,
 'currentTeam': None,
 'currentRosterStatus': 'RETIRED',
 'currentInjury': None,
 'height': '5\'9"',
 'weight': 170,
 'birthDate': '1988-04-30',
 'age': 30,
 'birthCity': 'Hamilton, ON',
 'birthCountry': 'Canada',
 'rookie': False,
 'highSchool': None,
 'college': None,
 'handedness': {'shoots': 'R'},
 'officialImageSrc': 'https://nhl.bamcontent.com/images/headshots/current/168x168/8476805@2x.jpg',
 'socialMediaAccounts': [],
 'currentContractYear': None,
 'drafted': None,
 'externalMappings': [{'source': 'NHL.com', 'id': 8476805}]}

In [33]:
#Calling our specific data we want
first_name = []
last_name = []
birthCity = []
birthCountry = []
team_position = []
currentRosterStatus = []
birthDate = []

for player in players:
 
        first_name.append(player["player"]["firstName"])
        last_name.append(player["player"]["lastName"])
        birthDate.append(player["player"]["birthDate"])
        birthCity.append(player["player"]["birthCity"])
        birthCountry.append(player["player"]["birthCountry"])
        team_position.append(player["player"]["primaryPosition"])
        currentRosterStatus.append(player["player"]["currentRosterStatus"])

    
    

In [35]:
#Creating DF
player_stats = {
    "Last_Name" : last_name,
    "First_Name" : first_name,
    "Birth_Date": birthDate,
    "Birth_City" : birthCity,
    "Birth_Country" : birthCountry,
    "Team_Position" : team_position,
    "Current_Roster_Status" : currentRosterStatus
}

player_stats_df = pd.DataFrame(player_stats)
player_stats_df.head()

,First_Name,Last_Name,Birth_City,Birth_Date,Birth_Country,Team_Position,Current_Roster_Status
0,Spencer,Abbott,"Hamilton, ON",1988-04-30,Canada,LW,RETIRED
1,Justin,Abdelkader,"Muskegon, MI",1987-02-25,USA,LW,ROSTER
2,Pontus,Aberg,Stockholm,1993-09-22,Sweden,LW,ROSTER
3,Noel,Acciari,"Johnston, RI",1991-12-01,USA,C,ROSTER
4,Will,Acton,"Edina, MN",1987-07-16,USA,C,RETIRED


In [46]:
#Connecting to engine
rds_connection_string = "root:Thelma21!!@127.0.0.1/hockey_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [47]:
#Checking table names in SQL
engine.table_names()

['injury', 'player_stats']

In [49]:
#Sending our Injury df to SQL
condensed_injury.to_sql(name="injury", con=engine, if_exists="append", index=False)

In [51]:
#Sending our Player Stats df to SQL
player_stats_df.to_sql(name="player_stats", con=engine, if_exists="append", index=False)